In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request
from timeit import default_timer as timer
import time
start = timer()
import re
url = "https://krisha.kz/prodazha/kvartiry/almaty/?page="
page = 1

list_of_links = []
print("Program started")
try:
    while page <= 1000:
        if page % 100 == 0:
            print("Loading process: ", int(page / 100) * 10, "%",sep = "")
        try:
             
            html_page = urllib.request.urlopen(url + str(page))
            
        except:
            page += 1
            break
        soup = BeautifulSoup(html_page, "html.parser")
        l = soup.findAll("div", {"class": "a-card__inc"})
        for l1 in l:
            x = l1.find("a", {"class": "a-card__image"}).get('href')
            list_of_links.append(x)
            print(x)
        page += 1
        
        time.sleep(0.5)
except KeyboardInterrupt:
    print('Loop was Interrupted by keyboard')
    
pd.DataFrame(list_of_links,columns = ['link']).to_excel(r'C:/Users/danii/Jupyter Notebook Study/BIS Project/list_of_links.xlsx', index = False, header=True)    
print("Loading already finished")
end = timer()
print("Count of notes:", len(list_of_links))    
print("Time of execution:", end - start, "seconds")

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request
from timeit import default_timer as timer
import time
import re
import errno
start = timer()
url = "https://krisha.kz"


list_of_links = pd.read_excel('list_of_links.xlsx')

Dataset = []
size = len(list_of_links)
print("Program started")

try:
    # I parsed the web site by 1500 flats at one approach with some time intervals,
    # on purpose to avoid the blocking of Http request.  
    for i in range(0, 1500):
        if i % 1926 == 0:
            print("Loading process: ", int(i / 1926) * 10, "%",sep = "")
        link = list_of_links['link'][i]
        try:
            html_page = urllib.request.urlopen(url + link)
            soup = BeautifulSoup(html_page, "html.parser")
        except Exception as e:
            print(e)
            continue
        
        print(link)
        
            
        
        item = []
        
        #Operaion 0, scraping the count of rooms
        l = soup.find("meta", {"name": "description"})
        x = int(re.findall('[0-9]+', l.get('content'))[0])
        if x < 20:
            item.append(x)
        else:
            l = soup.find("div", {"class": "gallery__main"}).find("a").find("picture").find("img").get('alt')
            x = int(re.findall('[0-9]+', l)[0])
            if x < 20:
                item.append(x)
            else:
                continue


        #Operaion 1, scraping the building type
        l = soup.find("div", {"data-name": "flat.building"})
        if l is not None:
            l2 = l.find("div", {"class": "offer__advert-short-info"})
            item.append(l2.text)
        else:
            item.append('?')
        #Operaion 2, scraping the flats floors and the count of levels at building
        l = soup.find("div", {"data-name": "flat.floor"})
        if l is not None:
            l2 = l.find("div", {"class": "offer__advert-short-info"})
            l3 = re.findall('[0-9]+', l2.text)
            if len(l3) == 2:
                item.append(int(l3[0]))
                item.append(int(l3[1]))
            else:
                item.append(0)
                item.append(int(l3[0]))
        else:
            item.append(0)
            item.append(0)
        #Operaion 3, scraping the square of rooms
        l = soup.find("div", {"data-name": "live.square"})
        l2 = l.find("div", {"class": "offer__advert-short-info"})
        l3 = re.findall('[0-9]+', l2.text)
        item.append(int(l3[0]))
        #Operaion 4, scraping the condition of flat
        l = soup.find("div", {"data-name": "flat.renovation"})
        if l is not None:
            l2 = l.find("div", {"class": "offer__advert-short-info"})
            item.append(l2.text)
        else:
            item.append('?')
        #Operaion 5, scraping the year of building
        l = soup.find("div", {"data-name": "house.year"})
        if l is not None:
            l2 = l.find("div", {"class": "offer__advert-short-info"})
            item.append(int(l2.text))
        else:
            item.append('?')
        #Operaion 6, scraping the balcony of flat
        l = soup.find("dt", {"data-name": "flat.balcony"})
        if l is not None:

            l2 = l.parent.find("dd")
            item.append(l2.text)
        else:
            item.append('?')

        #Operaion 7, scraping the furniture of flat
        l = soup.find("dt", {"data-name": "live.furniture"})
        if l is not None:

            l2 = l.parent.find("dd")
            item.append(l2.text)
        else:
            item.append('?')
        #Operaion 8, scraping the flooring of flat
        l = soup.find("dt", {"data-name": "flat.flooring"})
        if l is not None:

            l2 = l.parent.find("dd")
            item.append(l2.text)
        else:
            item.append('?')
        #Operaion 9, scraping the dormitory of flat
        l = soup.find("dt", {"data-name": "flat.priv_dorm"})
        if l is not None:

            l2 = l.parent.find("dd")
            item.append(l2.text)
        else:
            item.append('?')
        #Operaion 10, scraping the flat cost   
        l = soup.find("meta", {"name": "og:title"})
        if l is not None:

            item.append(int(re.findall('[0-9]+', l.get('content'))[-1]))
        else:
            item.append('?')
        Dataset.append(item)

        time.sleep(0.5)
except KeyboardInterrupt:
    print('Loop was Interrupted by keyboard')
    pass
# Dataset = pd.concat([pd.read_excel('dataset7500-9000.xlsx'), pd.DataFrame(Dataset, columns = ['Комнатность', 'Тип дома', 'Этаж', 'Этажность дома', 'Площадь, м²', 'Состояние', 'Год постройки', 'Балкон', 'Квартира меблирована', 'Пол', 'Бывшее общежитие', 'Стоимость'])], ignore_index=True)
pd.DataFrame(Dataset, columns = ['Комнатность', 'Тип дома', 'Этаж', 'Этажность дома', 'Площадь, м²', 'Состояние', 'Год постройки', 'Балкон', 'Квартира меблирована', 'Пол', 'Бывшее общежитие', 'Стоимость']).to_excel(r'C:/Users/danii/Jupyter Notebook Study/Projects/Krisha Prediction Model/dataset.xlsx', index = False, header=True)
# Dataset.to_excel('dataset7500-9000.xlsx', index = False, header=True)
print("Loading already finished")
end = timer()
print("Time of execution:", end - start, "seconds")
